<a href="https://colab.research.google.com/github/elichen/aoc2018/blob/main/Day_13_Mine_Cart_Madness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
data = r"""/->-\
|   |  /----\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/   """.split('\n')

In [ ]:
data = open("input.txt").read().split('\n')

In [ ]:
def process_tracks(track_map):
    # Initialize variables to store the cleaned map and cart positions
    cleaned_map = []
    carts = []

    # Iterate over each row in the track map
    for y, row in enumerate(track_map):
        # Create a list to store the cleaned row
        cleaned_row = list(row)

        # Iterate over each cell in the row
        for x, cell in enumerate(row):
            # Check for carts and their directions
            if cell in '^v':
                # Replace cart with a vertical track
                cleaned_row[x] = '|'
                # Store the cart position and direction
                carts.append((x, y, cell, 'L'))
            elif cell in '<>':
                # Replace cart with a horizontal track
                cleaned_row[x] = '-'
                # Store the cart position and direction
                carts.append((x, y, cell, 'L'))

        # Add the cleaned row to the cleaned map
        cleaned_map.append(''.join(cleaned_row))

    return cleaned_map, carts

cleaned_map, orig_carts = process_tracks(data)

In [ ]:
def move_carts_and_detect_crash(track_map, carts):
    direction_deltas = {'^': (0, -1), 'v': (0, 1), '<': (-1, 0), '>': (1, 0)}
    turn_left = {'^': '<', 'v': '>', '<': 'v', '>': '^'}
    go_straight = {'^': '^', 'v': 'v', '<': '<', '>': '>'}
    turn_right = {'^': '>', 'v': '<', '<': '^', '>': 'v'}

    carts.sort(key=lambda cart: (cart[1], cart[0]))
    updated_carts = []
    future_positions = {cart[:2] for cart in carts}  # Positions of all carts before moving

    for x, y, direction, next_turn in carts:
        dx, dy = direction_deltas[direction]
        new_x, new_y = x + dx, y + dy

        # Check for crashes against the positions of all carts (including those not yet moved)
        if (new_x, new_y) in future_positions:
            return updated_carts + [(new_x, new_y, 'X', next_turn)], (new_x, new_y)  # Return on crash

        # Update the set of future positions
        future_positions.remove((x, y))
        future_positions.add((new_x, new_y))

        track_type = track_map[new_y][new_x]
        new_direction = direction
        new_next_turn = next_turn

        if track_type == '+':
            if next_turn == 'L':
                new_direction = turn_left[direction]
                new_next_turn = 'S'
            elif next_turn == 'S':
                new_direction = go_straight[direction]
                new_next_turn = 'R'
            elif next_turn == 'R':
                new_direction = turn_right[direction]
                new_next_turn = 'L'

        elif track_type in '/\\':
            if track_type == '/':
                new_direction = '>' if direction == '^' else '<' if direction == 'v' else '^' if direction == '>' else 'v'
            else:  # track_type == '\\'
                new_direction = '<' if direction == '^' else '>' if direction == 'v' else '^' if direction == '<' else 'v'

        updated_carts.append((new_x, new_y, new_direction, new_next_turn))

    return updated_carts, None  # No crash occurred

In [ ]:
def print_map_with_carts(track_map, carts):
    # Convert the track map into a list of lists for easy manipulation
    map_with_carts = [list(row) for row in track_map]

    # Overlay the carts onto the map
    for x, y, direction, _ in carts:
        map_with_carts[y][x] = direction

    # Print the map
    for row in map_with_carts:
        print(''.join(row))

In [ ]:
carts = orig_carts.copy()
while True:
  # print_map_with_carts(cleaned_map, carts)
  carts, crash = move_carts_and_detect_crash(cleaned_map, carts)
  if crash:
    break
# print_map_with_carts(cleaned_map, carts)
crash

(124, 130)

In [ ]:
def move_carts_and_remove_on_crash(track_map, carts):
    direction_deltas = {'^': (0, -1), 'v': (0, 1), '<': (-1, 0), '>': (1, 0)}
    turn_left = {'^': '<', 'v': '>', '<': 'v', '>': '^'}
    go_straight = {'^': '^', 'v': 'v', '<': '<', '>': '>'}
    turn_right = {'^': '>', 'v': '<', '<': '^', '>': 'v'}

    carts.sort(key=lambda cart: (cart[1], cart[0]))
    updated_carts = []
    future_positions = {cart[:2] for cart in carts}  # Positions of all carts before moving
    crash_positions = set()  # To track positions of crashes

    for x, y, direction, next_turn in carts:
        if (x, y) in crash_positions:  # Skip carts that have crashed
            continue

        dx, dy = direction_deltas[direction]
        new_x, new_y = x + dx, y + dy

        if (new_x, new_y) in future_positions:
            # Remove both carts involved in the crash
            updated_carts = [cart for cart in updated_carts if cart[:2] != (new_x, new_y)]
            future_positions.remove((new_x, new_y))
            crash_positions.add((new_x, new_y))
            continue

        future_positions.remove((x, y))
        future_positions.add((new_x, new_y))

        track_type = track_map[new_y][new_x]
        new_direction = direction
        new_next_turn = next_turn

        if track_type == '+':
            if next_turn == 'L':
                new_direction = turn_left[direction]
                new_next_turn = 'S'
            elif next_turn == 'S':
                new_direction = go_straight[direction]
                new_next_turn = 'R'
            elif next_turn == 'R':
                new_direction = turn_right[direction]
                new_next_turn = 'L'

        elif track_type in '/\\':
            if track_type == '/':
                new_direction = '>' if direction == '^' else '<' if direction == 'v' else '^' if direction == '>' else 'v'
            else:  # track_type == '\\'
                new_direction = '<' if direction == '^' else '>' if direction == 'v' else '^' if direction == '<' else 'v'

        updated_carts.append((new_x, new_y, new_direction, new_next_turn))

    return updated_carts

In [ ]:
# data = r"""/>-<\
# |   |
# | /<+-\
# | | | v
# \>+</ |
#   |   ^
#   \<->/""".split('\n')
# cleaned_map, orig_carts = process_tracks(data)

In [ ]:
carts = orig_carts.copy()
while True:
  carts = move_carts_and_remove_on_crash(cleaned_map, carts)
  if len(carts) <= 1:
    break
# print_map_with_carts(cleaned_map, carts)
carts

[(143, 123, '^', 'R')]